In [1]:
with open('config/pg_conn.txt') as conn_details:
    conn_str_psyco = conn_details.readline()
    conn_str_sqlalchemy = conn_details.readline()

import pandas as pd
import pandas.io.sql as psql
import psycopg2 as pg
from sqlalchemy import create_engine
import sqlalchemy as sa
import os
import glob

In [2]:
engine = create_engine(conn_str_sqlalchemy)

In [5]:
SPEC_SCHEMA = 'dhs_survey_specs'
DATA_SCHEMA = 'dhs_data_tables'

#TABLESPEC_TABLENAME = 'table_specs_blank' #'dhs_table_specs_flat'
TABLESPEC_TABLENAME = 'dhs_table_specs_flat'
#VALUESPEC_TABLENAME = 'value_specs_blank' #'dhs_value_descs'
VALUESPEC_TABLENAME = 'dhs_value_descs'
#SURVEYLIST_TABLENAME = 'dhs_survey_listing_backup'
SURVEYLIST_TABLENAME = 'dhs_survey_listing'

TABLE_SPEC_TABLE = ".".join((SPEC_SCHEMA, TABLESPEC_TABLENAME))
VALUE_SPEC_TABLE = ".".join((SPEC_SCHEMA, VALUESPEC_TABLENAME))
SURVEYLIST_TABLE = ".".join((SPEC_SCHEMA, SURVEYLIST_TABLENAME))

STAGING_FOLDER = "/mnt/d/InformalCities/DHS_Updates/staging"

In [6]:
table_name = "REC01"

In [ ]:
 surveyid character varying(3) COLLATE pg_catalog."default",
    caseid character varying(18) COLLATE pg_catalog."default",
    v000 character varying(3) COLLATE pg_catalog."default",
    v001 character varying(8) COLLATE pg_catalog."default",

In [51]:
engine.execute?

Signature: engine.execute(statement, *multiparams, **params)
Docstring:
Executes the given construct and returns a
:class:`_engine.ResultProxy`.

The arguments are the same as those used by
:meth:`_engine.Connection.execute`.

Here, a :class:`_engine.Connection` is acquired using the
:meth:`_engine.Engine.contextual_connect` method,
and the statement executed
with that connection. The returned :class:`_engine.ResultProxy`
is flagged
such that when the :class:`_engine.ResultProxy` is exhausted and its
underlying cursor is closed, the :class:`_engine.Connection`
created here
will also be closed, which allows its associated DBAPI connection
resource to be returned to the connection pool.
File:      ~/anaconda3/envs/geodev_38/lib/python3.8/site-packages/sqlalchemy/engine/base.py
Type:      method


In [62]:
r = engine.execute("""SELECT
    tablename,
    indexname,
    indexdef
FROM
    pg_indexes
WHERE
    schemaname = 'dhs_data_tables'
ORDER BY
    tablename,
    indexname;""")

In [65]:
engine.raw_connection?

Signature: engine.raw_connection(_connection=None)
Docstring:
Return a "raw" DBAPI connection from the connection pool.

The returned object is a proxied version of the DBAPI
connection object used by the underlying driver in use.
The object will have all the same behavior as the real DBAPI
connection, except that its ``close()`` method will result in the
connection being returned to the pool, rather than being closed
for real.

This method provides direct DBAPI connection access for
special situations when the API provided by
:class:`_engine.Connection`
is not needed.   When a :class:`_engine.Connection` object is already
present, the DBAPI connection is available using
the :attr:`_engine.Connection.connection` accessor.

.. seealso::

    :ref:`dbapi_connections`
File:      ~/anaconda3/envs/geodev_38/lib/python3.8/site-packages/sqlalchemy/engine/base.py
Type:      method


In [191]:
db_helper = TableDataHelper(conn_str=conn_str_sqlalchemy, table_spec_table=TABLESPEC_TABLENAME,
                             value_spec_table=VALUESPEC_TABLENAME, spec_schema=SPEC_SCHEMA,
                           data_schema=DATA_SCHEMA, dry_run=True)

In [192]:
db_helper._does_data_table_exist("RECHEM")

True

In [193]:
db_helper._is_dry_run

True

In [163]:
db_helper._is_dry_run = False

In [194]:
r = db_helper.create_data_table("RECHEM")
print(r)

Skipped existing index surveyid_rechem
Skipped existing index hhid_rechem
Skipped existing index idxhe_rechem
Skipped existing covering index allidx_rechem
Skipped existing seoncdary covering index twoidx_rechem
Would create table with 

            CREATE TABLE dhs_data_tables."RECHEM"(surveyid character varying(3) COLLATE pg_catalog."default",
hhid character varying(12) COLLATE pg_catalog."default",
idxhe character varying(2) COLLATE pg_catalog."default",
sh63 character varying(1) COLLATE pg_catalog."default",
sh64 character varying(1) COLLATE pg_catalog."default",
sh65 character varying(1) COLLATE pg_catalog."default",
sh66 character varying(1) COLLATE pg_catalog."default",
sh67 character varying(1) COLLATE pg_catalog."default",
sh68 character varying(2) COLLATE pg_catalog."default",
sh69 character varying(3) COLLATE pg_catalog."default",
sh70 character varying(3) COLLATE pg_catalog."default",
sh71 character varying(1) COLLATE pg_catalog."default",
sh72 character varying(1) COLLATE 

In [195]:
r = engine.execute(f'SELECT * FROM dhs_data_tables."RECH1" LIMIT 0')

In [204]:
pd.read_sql(f"""
            SELECT LOWER(name) AS name, MAX(len) AS maxlen, MAX(start) as start
            FROM {TABLE_SPEC_TABLE}
            WHERE recordname = '{table_name}' AND lower(name) LIKE '%id%'
            GROUP BY name""", con=engine)
        

,name,maxlen,start
0,aidsex,1,157
1,caseid,18,1


In [184]:
whats_needed.loc[-1] = ('surveyid', 'Item', '3', 0)
whats_needed.index = whats_needed.index + 1  # shifting index
whats_needed.sort_index(inplace=True) 

In [186]:
whats_needed.loc[len(whats_needed)] = ('data', 'Item', '', 999999999)

In [187]:
whats_needed

,name,itemtype,length,start
0,surveyid,Item,3,0
1,caseid,IdItem,18,1
2,v001,Item,8,34
3,v002,Item,6,42
4,v003,Item,3,46
5,v004,Item,4,49
6,v005,Item,8,53
7,v008a,Item,5,59
8,v006,Item,2,61
9,v007,Item,4,63


In [190]:
import sqlalchemy as sa
from sqlalchemy import create_engine
import pandas as pd
import warnings
import os

class TableDataHelper:
    _MAX_COLUMN_THRESHOLD = 1000
    def __init__(self, conn_str, 
        table_spec_table, value_spec_table, spec_schema,
        data_schema, dry_run=True):
        self._engine = create_engine(conn_str)
        self._TABLE_SPEC_TABLENAME = table_spec_table
        self._VALUE_SPEC_TABLENAME = value_spec_table
        self._SPEC_SCHEMA = spec_schema
        self._TABLE_SPEC_TABLE = ".".join([spec_schema, table_spec_table])
        self._VALUE_SPEC_TABLE = ".".join([spec_schema, value_spec_table])
        self._DATA_SCHEMA = data_schema
        self._is_dry_run = dry_run
        
        self._populate_JSON_table_list()
        
        self._known_tables = set()
        self._checked_column_tables = set()

    
    def _populate_JSON_table_list(self):
        json_tables = pd.read_sql(f"""
            SELECT DISTINCT table_name 
            FROM information_schema.columns 
            WHERE table_schema = '{self._DATA_SCHEMA}' AND data_type = 'jsonb'""",
                con=self._engine)
        self._json_tables = set(json_tables['table_name'])

    def _does_data_table_exist(self, table_name):
        if table_name in self._known_tables:
            return True
        exists = pd.read_sql(f"""
            SELECT EXISTS (
                SELECT FROM information_schema.tables 
                WHERE  table_schema = '{self._DATA_SCHEMA}'
                AND    table_name   = '{table_name}'
            );""", con=self._engine)['exists'][0]
        if exists:
            self._known_tables.add(table_name)
            return True
        return False


    def prepare_db_for_file(self, table_name):
        if not self._does_data_table_exist(table_name):
            self.create_data_table(table_name)
        else:
            self.check_cols_against_metadata(table_name)


    def create_data_table(self, table_name):
        """Creates a data table with all the columns that are currently specified in the metadata.
        
        As throughout this software, all columns are created with VARCHAR(n) type where n = the 
        maximum width currently specified for that table/column in any survey
        
        The exception if there are more than TableDataHelper._MAX_COLUMN_THRESHOLD columns 
        specified in the metadata, in which case only columns with "id" in the name will be 
        first-class columns, and a jsonb column named data will be added for storing the remainder."""
        column_clauses, is_json = self._get_column_clauses(table_name)
        create_stmt = f"""
            CREATE TABLE {self._DATA_SCHEMA}."{table_name}"({column_clauses})
            TABLESPACE pg_default;
            ALTER TABLE {self._DATA_SCHEMA}."{table_name}" OWNER to admin;"""
        if self._is_dry_run:
            return ("Would create table with \n" + create_stmt + "\n" 
            + " then create indices with " + self._create_or_replace_indices(table_name, is_json))
        else:
            r = self._engine.execute(create_stmt)
            index_sql = self._create_or_replace_indices(table_name, is_json)
            if len(index_sql) > 0:
                print("Executing the following to drop / recreate indices: \n" + index_sql)
                self._engine.execute(index_sql)
        

    def _get_column_clauses(self, table_name):
        """Gets the columns that a new data table should have, according to the metadata tables.
        
        Returns them as a string for use in a statement of the form 
        CREATE TABLE tablename (result) """
        
        # list all columns that are specified for this datatable in the survey metadata (unioned 
        # set across all surveys: not all surveys will have all columns)
        whats_needed = pd.read_sql(f"""
            SELECT lower(name) AS name, itemtype, MAX(len) AS length, MAX(start) AS start
            FROM {self._TABLE_SPEC_TABLE}
            WHERE recordname='{table_name}'
            GROUP BY name, itemtype
            ORDER BY start;
        """, con=self._engine)
        
        # In the case of some country-specific tables, where the columns are different in almost 
        # every survey, the number of columns becomes very large and horribly inefficient to store 
        # (it is sparse). In these cases we store those tables with a single JSONB column for the data 
        # plus the ID/joining columns as first-class columns
        is_json = False
        if len(whats_needed) > TableDataHelper._MAX_COLUMN_THRESHOLD:
            whats_needed = whats_needed[whats_needed['name'].str.contains('id')]
            whats_needed.loc[len(whats_needed)] = ('data', 'JSON', '', 99999999)
            is_json=True

        # of course the metadata tables don't specify surveyid so we add that manually
        whats_needed.loc[-1] = ('surveyid', 'Item', '3', 0)
        whats_needed.index = whats_needed.index + 1
        whats_needed.sort_index(inplace=True)
        
        def _clause_from_row(r):
            if r["itemtype"] == "JSON":
                type_clause = "jsonb"
            else:
                type_clause = f'character varying({r["length"]})'
            clause = f'{r["name"]} {type_clause} COLLATE pg_catalog."default"'
            return clause 

        # convert each row in the df to a clause for use in the CREATE TABLE statement
        clauses = list(whats_needed.apply(_clause_from_row, axis=1))
        return (",\n".join(clauses), is_json)


    def _create_or_replace_indices(self, table_name, is_json=False, replace_existing=False):
        # TODO create GIN index on any JSON columns?
        idx_sql_template = 'CREATE INDEX {0} ON {1}."{2}"({3});'
        idx_name_template = '{0}_{1}'
        clean_sql_template = 'DROP INDEX IF EXISTS {0}.{1};'

        res = self._engine.execute("SELECT relname FROM pg_class WHERE relkind='i';")
        existing_indices = [i[0] for i in res.fetchall()]

        tbl_cols = self._engine.execute(f'SELECT * FROM dhs_data_tables."{table_name}" LIMIT 0').keys()
        idx_fields = [c for c in tbl_cols if c.find('id') != -1]

        drop_idx_stmts = []
        idx_stmts = []

        for c in idx_fields:
            idx_name = idx_name_template.format(c, str.lower(table_name))
            idx_sql = idx_sql_template.format(idx_name, self._DATA_SCHEMA, table_name, c)
            if idx_name in existing_indices:
                if replace_existing:
                    drop_idx_stmt = clean_sql_template.format(self._DATA_SCHAME, idx_name)
                    drop_idx_stmts.append(drop_idx_stmt)
                    idx_stmts.append(idx_sql)
                    print("Replacing index " + idx_name)
                else:
                    print("Skipped existing index " + idx_name)
            else:
                idx_stmts.append(idx_sql)
                print("Adding index "+idx_name)
        
        # also create a single covering index on all joining columns
        if len(idx_fields) > 1:
            idx_name = idx_name_template.format("allidx", str.lower(table_name))
            idx_sql = idx_sql_template.format(idx_name, self._DATA_SCHEMA, table_name, ",".join(idx_fields))
            if idx_name in existing_indices:
                if replace_existing:
                    drop_idx_stmt = clean_sql_template.format(self._DATA_SCHEMA, idx_name)
                    drop_idx_stmts.append(drop_idx_stmt)
                    idx_stmts.append(idx_sql)
                    print("Replacing covering index " + idx_name)
                else:
                    print("Skipped existing covering index " + idx_name)
            else:
                idx_stmts.append(idx_sql)
                print("Adding covering index "+idx_name)
        
        # also create a covering index on the first two joining columns if there are three 
        # (or all except the last one, if there's more)
        # e.g. surveyid and caseid but not bidx (the cols are in the appropriate order in the CSVs)
        if len(idx_fields) > 2:
            idx_name = idx_name_template.format("twoidx", str.lower(table_name))
            idx_sql = idx_sql_template.format(idx_name, self._DATA_SCHEMA,
                                            table_name, ",".join(idx_fields[:-1]))
            if idx_name in existing_indices:
                if replace_existing:
                    drop_idx_stmt = clean_sql_template.format(self._DATA_SCHEMA, idx_name)
                    drop_idx_stmts.append(drop_idx_stmt)
                    idx_stmts.append(idx_sql)
                    print ("Replacing secondary covering index " + idx_name)
                else:
                    print ("Skipped existing seoncdary covering index " + idx_name)
            else:
                idx_stmts.append(idx_sql)
                print ("Adding secondary covering index " + idx_name)
        
        drop_indices_sql = "\n".join(drop_idx_stmts)
        create_indices_sql = "\n".join(idx_stmts)

        return drop_indices_sql + "\n" + create_indices_sql
  

    def get_db_survey_table_rowcount(self, surveyid, tablename):
        pass


    def check_and_update_column_width(self, schema_name, table_name, column_name, 
                                req_width, update_if_needed=False):
        """Check that a varchar(n) column in the database is at least req_width wide and widen it if not"""
        df = pd.read_sql(f'''
            SELECT character_maximum_length as maxlen 
            FROM information_schema.columns 
            WHERE table_name = '{table_name}' and column_name = '{column_name}';''', con=self._engine)
        current = df['maxlen'][0]
        if current < req_width and update_if_needed:
            print(f"Column {schema_name}.{table_name}.{column_name} will be widened from"+
                "{current} to {req_width}")
            sql = f'''ALTER TABLE {schema_name}."{table_name}" 
                        ALTER COLUMN {column_name} TYPE character varying({req_width})'''
            return self._engine.execute(sql)
        else:
            return current >= req_width


    def check_destination_col_widths(self, df, schema_name, table_name):
        """Check that for all object columns in the provided dataframe, the equivalent column 
        in the database is at least wide enough to hold it, and widen it if not."""
        obj_cols = df.select_dtypes(include=['object']).columns
        lengths = {col:(df[col].str.len().max()) for col in obj_cols}
        for col, incoming_length in lengths.items():
            self.check_and_update_column_width(schema_name, table_name, col, incoming_length, True)

    
    def check_missing_cols(self, table_name):
        """For a given data table, check the metadata tables to see what all the columns 
        needed are, and then check whether they all exist. Returns a dataframe of column names 
        that are missing from the table, if the table is not one with a JSONB column."""
        
        data_cols_needed = pd.read_sql(f"""
            SELECT recordname, LOWER(name) AS name, MAX(len) AS maxlen 
            FROM {self._SPEC_SCHEMA}.{self._TABLE_SPEC_TABLENAME}
            WHERE recordname != '*'
            GROUP BY recordname, name
            ORDER BY recordname, name""", con=self._engine)
        
        data_cols_present = pd.read_sql(f"""
            SELECT table_name, column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = '{self._DATA_SCHEMA}' ORDER BY table_name, column_name""", 
            con=self._engine)
        
        data_cols_not_present = data_cols_needed[~data_cols_needed['name'].isin(
            data_cols_present['column_name'])]

        and_not_in_json_tbl = data_cols_not_present[~data_cols_not_present['recordname'].isin(
            self._json_tables)]
        
    

        

    def add_varchar_col_to_table(self, schema_name, table_name, column_name, req_width):
        sql = f"""
            ALTER TABLE {schema_name}."{table_name}" 
            ADD COLUMN {column_name.lower()} CHARACTER VARYING ({req_width})"""
        return engine.execute(sql)


    def delete_table_entries_for_survey(surveyid, table_name, dry_run=True):
        # only drop ones for matching columns because we get different ones from the IR and MR files
        if dry_run:
            print(f"Would drop all data rows for survey {surveyid} from {table_name}")
        else:
            print(f"Dropping all data rows for survey {surveyid} from {table_name}")
            meta = sa.MetaData()
            tablespec_db = sa.Table(table_name, meta, schema=self._DATA_SCHEMA, 
                                    autoload=True, autoload_with=engine)
            cond = tablespec_db.c.surveyid == surveyid
            delete = tablespec_db.delete().where(cond)  
            res = engine.execute(delete)